In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

# import

In [ ]:
# export
from exp import nb_resnet_ssd_std

In [ ]:
# export
import sys 
sys.path.append("..") 
#from exp import nb_bbox_hw_statistics
from detect_symbol.exp import nb_databunch
from detect_symbol.exp import nb_init_model
from detect_symbol.exp import nb_resnet_ssd
from detect_symbol.exp import nb_anchors_loss_metrics

In [ ]:
# export
import torch

In [ ]:
# export
from torch.nn import functional as F

In [ ]:
# export
from torch import tensor

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
# export
from IPython.core import debugger as idb

In [ ]:
# export
import numpy as np

In [ ]:
# export
import math

## functions

In [ ]:
def find_neibs(idx, grids = (49, 49), dis = 1, dbg = None):
    '''
    找到某个anchor周围相邻的anchors的下标里列表。距离默认1。
    这个任务中只有第一层的grids参与，所以只需要第一次的grids的尺寸。
    anchor也是1对1的。
    参数：
        idx：目标anchor在grid anchors(get_grid_anchors返回的gvs)列表中的下标
        grids: 尺寸
        dis：邻居的距离
    返回值：
        邻居的下标列表
    '''
    gh, gw = grids
    x = idx % gh
    y = idx // gw
    ret = []
    for nx in range(x - dis, x + dis + 1):
        for ny in range(y - dis, y + dis + 1):
            if nx >= 0 and ny >= 0 and nx < gw and ny < gh \
                    and not(nx == x and ny == y):
                nidx = ny * gw + nx
                if dbg:
                    print(x, y, nx, ny, nidx)
                ret += [nidx]
    return ret      
                
    

In [ ]:
# export
#定义一个新的GridAnchor_Functions，主要是修改get_scroe_hits
class GridAnchor_Funcs_std(nb_anchors_loss_metrics.GridAnchor_Funcs):
    def __init__(self, fig_hw, grids, device):
        anchors = [[(0, 0)]]
        gvs,ghs,gws,avs,ahs,aws = nb_anchors_loss_metrics.get_grids_anchors( \
                    fig_hw, grids, anchors)
        self.grids = grids
        super().__init__(gvs, avs, device)
        
    def get_scores_hits(self, gt_bboxs): 
        # ground truch bbox center x,y
        gt_cx = gt_bboxs[:,[0,2]].mean(-1)
        gt_cy = gt_bboxs[:,[1,3]].mean(-1)

        # 判断目标bbox的中心落在哪个cell内
        hits = ((gt_cx[:,None] >= self.gvs[:,0][None]) &
                (gt_cx[:,None] <  self.gvs[:,2][None]) &
                (gt_cy[:,None] >= self.gvs[:,1][None]) &
                (gt_cy[:,None] <  self.gvs[:,3][None]))
        
        # 在的就是1，不在就是0
        scores = hits * 1
        
        return scores,hits
    
    
    def t2b(self,t,idx,eps=1):
        cx,cy = self.gvs[idx,0],self.gvs[idx,1]
        gh,gw = self.ghs[idx],self.gws[idx]
        ph,pw = self.ahs[idx],self.aws[idx]

        sig_tx = torch.sigmoid(t[...,0])
        sig_ty = torch.sigmoid(t[...,1])
        #anchor可以设置1或者0，对应的数据集里面的也应该设置对应的值
        #或者干脆没有宽高信息
        exp_th = torch.zeros(idx.numel())
        exp_tw = torch.zeros(idx.numel())

        hatsig_tx = (1+eps)*(sig_tx-0.5) + 0.5
        hatsig_ty = (1+eps)*(sig_ty-0.5) + 0.5

        bx = hatsig_tx*gw + cx # x of center of box
        by = hatsig_ty*gh + cy # y of center of box

        bh = ph * exp_th    # height of box
        bw = pw * exp_tw    # width of box

        tl_x = bx - bh/2 # x of top-left corner of box
        tl_y = by - bw/2 # y of top-left corner of box 
        br_x = bx + bh/2 # x of bottom-right corner of box
        br_y = by + bw/2 # y of bottom-right corner of box

        res = torch.stack([tl_x, tl_y, br_x, br_y],dim=0)
        res = res.permute(list(range(len(res.shape)))[1:]+[0])
        return res
    
    def b2t(self, b,idx,eps=1):
        cx,cy = self.gvs[idx,0],self.gvs[idx,1]
        gh,gw = self.ghs[idx],self.gws[idx]
        ph,pw = self.ahs[idx],self.aws[idx]

        bx = (b[:,0] + b[:,2])/2 # x of center of box
        by = (b[:,1] + b[:,3])/2 # y of center of box
        bh = b[:,2] - b[:,0]     # height of box
        bw = b[:,3] - b[:,1]     # width of box

        hatsig_tx = (bx - cx)/gh
        hatsig_ty = (by - cy)/gw
        #anchor可以设置1或者0，对应的数据集里面的也应该设置对应的值
        #或者干脆没有宽高信息
        #exp_th = torch.zeros(idx.numel())
        #exp_tw = torch.zeros(idx.numel())

        sig_tx = (hatsig_tx+0.5*eps)/(1+eps)
        sig_ty = (hatsig_ty+0.5*eps)/(1+eps)

        tx = torch.log(sig_tx/(1-sig_tx))
        ty = torch.log(sig_ty/(1-sig_ty))
        #th = torch.log(exp_th)
        #tw = torch.log(exp_tw)
        th = torch.zeros(idx.numel())
        tw = torch.zeros(idx.numel())

        return torch.stack([tx, ty, th, tw]).t()
    

In [ ]:
# export
def yolo_L_std(pred_batch, *gt_batch, conf_th=0.5,
           lambda_cent=1, lambda_pconf=1, lambda_nconf=1, lambda_clas=1, clas_weights=None, gaf):
    '''
    与detect_symbol里面的yolo_L相比的区别是：
        不计算宽高方面的损失
        neg_idx要去掉find_neibs返回的discard列表
        
    clas_weights: 
    为了解决数据集的imbalance问题，一种方法是在dataloader中使用WeightedRandomSampler，但是这种方法不适用于目标检测问题。
    因为，（1）目标检测的label不是一个简单的数值（2）目标检测问题的一张图片可能包括不同类别的多个目标。
    所以为了解决目标检测问题中的imbalance问题，我们的方法是在损失函数中使用权重。
    为各类别分配权重，各目标对应的损失乘以该目标所属类别的权重。
    默认为None，即不使用权重。
    若设置非None，则clas_weights应该是一个一维tensor，其长度等于数据集的类别数。
    若设置为全1，则相当于不使用权重。
    合理的设置应保证所有元素之和等于数据集的类别数，否则相当于对损失函数的整体做了缩放。
    '''
    clas_loss = 0
    cent_loss = 0
    pconf_loss = 0
    nconf_loss = 0
    pos_cnt = 0
    neg_cnt = 0
    
    for pred_txy,pred_conf,pred_clas,gt_bboxs,gt_clas in zip(*pred_batch, *gt_batch):
        keep = nb_anchors_loss_metrics.get_y(gt_bboxs)
        if keep.numel()==0: continue
          
        gt_bboxs = gt_bboxs[keep]
        gt_clas = gt_clas[keep]
        
        gt_bboxs = (gt_bboxs + 1) / 2
        gt_clas = gt_clas - 1 # the databunch add a 'background' class to classes[0], but we don't want that,so gt_clas-1
        
        if clas_weights is not None: ws = clas_weights[gt_clas]
        else: ws = None
        
        scores,hits = gaf.get_scores_hits(gt_bboxs)
        idx = nb_anchors_loss_metrics.idx_fromScoresHits(scores,hits)
        
        # classification loss
        pred_clas = pred_clas[idx]
        clas_loss += F.cross_entropy(pred_clas, gt_clas, weight=clas_weights, reduction='sum')
        #import pdb; pdb.set_trace()
        # bbox center loss
        gt_t = gaf.b2t(gt_bboxs,idx,eps=1)
        pred_txy = pred_txy[idx]
        if ws is not None:
            cent_loss += ((gt_t[...,:2]-pred_txy)*ws[...,None]).abs().sum()
        else:
            cent_loss += (gt_t[...,:2]-pred_txy).abs().sum()
        
        # positive confidence loss
        conf_pos = pred_conf[idx]
        if ws is not None: 
            pconf_loss += F.binary_cross_entropy_with_logits(conf_pos,torch.ones_like(conf_pos),weight=ws[...,None],reduction='sum')
        else: 
            pconf_loss += F.binary_cross_entropy_with_logits(conf_pos,torch.ones_like(conf_pos),reduction='sum')
        #import pdb; pdb.set_trace()
        # negative conficence loss
        scores[range(0,len(idx)),idx] = conf_th + 1 # 强制责任anchor的得分好于threshold
        
        #这里把邻居也的conf也调高。后面就被过滤掉了
        #scores[range(0,len(discards)),discards] = conf_th + 1
                
        tmp = scores>conf_th # 判断各得分是否好于threshold
        tmp = tmp.max(dim=0)[0] # 各anchor是否有任意一个好于threshold的得分
        
        #取得命中的anchor周围的anchor的下标立标
        discards = []
        #print('tofind', idx)
        for hidx in idx:
            #gaf.grids:[(49,49)]
            #print('finding', hidx)
            neibs = find_neibs(hidx, gaf.grids[0], dis = 1)            
            for i in neibs:
                discards += [i]
        #print('discards:', discards)
        #import pdb; pdb.set_trace()        
        tmp[discards] = 1
         
        neg_idx = torch.where(tmp==0)[0] # 如果没有，该anchor是negative anchor
        
        conf_neg = pred_conf[neg_idx]
        nconf_loss += F.binary_cross_entropy_with_logits(conf_neg,torch.zeros_like(conf_neg),reduction='sum')
        
        pos_cnt += len(idx)
        neg_cnt += len(neg_idx)
        
    clas_loss  = lambda_clas  * clas_loss  /pos_cnt
    cent_loss  = lambda_cent  * cent_loss  /pos_cnt
    pconf_loss = lambda_pconf * pconf_loss /pos_cnt
    nconf_loss = lambda_nconf * nconf_loss /neg_cnt
    print('yolo_l', clas_loss, cent_loss, pconf_loss, nconf_loss)
    
    return clas_loss + cent_loss + pconf_loss + nconf_loss


In [ ]:
# export
def bbox2chw_std(b):
    '''
    将bbox的（左上x，左上y，右下x，右下y）表示变为（中心x，中心y，高，宽）表示
    '''
    cx = b[...,[0,2]].mean(-1)[...,None]
    cy = b[...,[1,3]].mean(-1)[...,None]
    
    h = torch.zeros_like(cx)
    w = torch.zeros_like(cx)
    
    return torch.cat([cx,cy,h,w],dim=-1)

In [119]:
# export
def cent_d_std(pred_batch, *gt_batch, gaf):
    #import pdb; pdb.set_trace()
    dif = tensor(0.)
    cnt = tensor(0.)
    for pred_txy,gt_bboxs,_ in zip(pred_batch[0], *gt_batch):
        keep = nb_anchors_loss_metrics.get_y(gt_bboxs)
        if keep.numel()==0: continue
          
        #pred_t = torch.cat([pred_txy,pred_thw],dim=1)
        pred_t = pred_txy
        
        gt_bboxs = gt_bboxs[keep]
        gt_bboxs = (gt_bboxs + 1) / 2
        
        scores,hits = gaf.get_scores_hits(gt_bboxs)
        idx = nb_anchors_loss_metrics.idx_fromScoresHits(scores,hits)
        
        pred_t = pred_t[idx]
        pred_c = bbox2chw_std(gaf.t2b(pred_t,idx))[...,:2]
        gt_c = bbox2chw_std(gt_bboxs)[...,:2]
        
        tmp = (gt_c - pred_c).abs().sum()
        dif += tmp
        cnt += len(idx)
    
    return dif/cnt/2

# test

In [ ]:
#get_grid_anchors的生成顺序
grids = [(49,49),(25,25),(13,13)]
anchors = [[(22,17),(22,37),(43,17),(43,37)],
            [(43,77),(83,37),(83,77)],
            [(83,162),(162,77),(162,162)]]
t = [(x, y, gx, gy, ax, ay) for (gx,gy),ancs in zip(grids,anchors)
                                for y in range(gy)
                                for x in range(gx)
                                for ax,ay in ancs]
print(t[:100])

In [ ]:
#测试find_neibs
neibs = find_neibs(100, (49, 49))

gvs,ghs,gws,avs,ahs,aws = nb_anchors_loss_metrics.get_grids_anchors( \
                                (776,776), [(49,49)], [[(1,1)]])
print('邻居', neibs)
for idx in neibs:
    print(idx, anchors)
    print(avs[idx])

In [ ]:
#距离之间的事情可以不考虑？先按照resnet18来一个
# 获取 grids 和 anchors
gvs,ghs,gws,avs,ahs,aws = nb_anchors_loss_metrics.get_grids_anchors(
                                            fig_hw = (776,776),
                                            grids = [(49,49)],
                                            anchors = [[(1,1)]])

In [ ]:
# 这些变量的尺寸
gvs.shape, ghs.shape, gws.shape, avs.shape, ahs.shape, aws.shape

In [ ]:
49*49*1  #只有一个anchor

In [ ]:
device = torch.device('cuda')

In [ ]:
device = torch.device('cpu')

In [ ]:
device_ids = [0, 1]

In [ ]:
gaf = GridAnchor_Funcs_std(fig_hw = (776,776)
                         , grids = [(49,49)]
                         , device = device)

In [ ]:
#先用detect_symbol的数据集测试
data = nb_databunch.get_databunch(data_root = '../detect_symbol/data/ds_20200429', bs=4,device=device)
x,y = data.one_batch()

In [133]:
# get model and load init parameters
#用符号检测的数据集测试!!!!!!类别不同!!!!
model = nb_resnet_ssd_std.get_resnet18_1ssd_std(num_classes = 17)


In [143]:
#model.load_state_dict(torch.load('./models/resnet18_1ssd_init.pth'));
model.load_state_dict(torch.load('./models/pretrained_res18_1ssd.pth'));

In [ ]:
# check the statistics of layer outputs
#nb_init_model.show_layer_stats(model,x)

In [ ]:
#x = x.to(device)

In [144]:
# pred a batch

pred = model(x)

### find idx of the responsible anchor

In [ ]:
y[1].shape

In [ ]:
# 选择一个bbox
samp = 1 # samp: sample, get one bbox from this sample
bb = (y[0][samp][-1]+1)/2
bb

In [ ]:
def dbg():
    import pdb; pdb.set_trace()
    #gaf = GridAnchor_Funcs_std(fig_hw = (776,776)
    #                     , grids = [(49,49)]
    #                     , device = device)
    scores,hits= gaf.get_scores_hits(bb.to(device)[None])
    idx = idx_fromScoresHits(scores,hits)
    print(idx)
#dbg()    

In [ ]:
# 测试 get_scores_hits 和 idx_fromScoresHits
scores,hits= gaf.get_scores_hits(bb.to(device)[None])
#idx = idx_fromScoresHits(scores,hits)
#idx

### losses

In [128]:
# classification loss
def dbg():
    import pdb; pdb.set_trace()
    loss = nb_anchors_loss_metrics.cent_L(pred,*y,gaf=gaf, clas_weights=None)
    return loss
loss = dbg()
loss

> <ipython-input-128-9418facdd748>(4)dbg()
-> loss = nb_anchors_loss_metrics.cent_L(pred,*y,gaf=gaf, clas_weights=None)
(Pdb) s
--Call--
> /home/dev/jupyter/detect_symbol/exp/nb_anchors_loss_metrics.py(261)cent_L()
-> def cent_L(pred_batch, *gt_batch, lambda_cent=1, clas_weights=None, gaf):
(Pdb) l
256  	
257  	    return lambda_clas*loss/cnt
258  	
259  	
260  	#================================================
261  ->	def cent_L(pred_batch, *gt_batch, lambda_cent=1, clas_weights=None, gaf):
262  	    loss = 0
263  	    cnt = 0
264  	    for pred_txy,gt_bboxs,gt_clas in zip(pred_batch[0], *gt_batch):
265  	        keep = get_y(gt_bboxs)
266  	        if keep.numel()==0: continue
(Pdb) n
> /home/dev/jupyter/detect_symbol/exp/nb_anchors_loss_metrics.py(262)cent_L()
-> loss = 0
(Pdb) 
> /home/dev/jupyter/detect_symbol/exp/nb_anchors_loss_metrics.py(263)cent_L()
-> cnt = 0
(Pdb) 
> /home/dev/jupyter/detect_symbol/exp/nb_anchors_loss_metrics.py(264)cent_L()
-> for pred_txy,gt_bboxs,gt_clas 

tensor(19.4417, grad_fn=<DivBackward0>)

In [145]:

# classification loss
clas_loss = nb_anchors_loss_metrics.clas_L(pred,*y,gaf=gaf, clas_weights=None)
clas_loss

tensor(4.4277, grad_fn=<DivBackward0>)

In [146]:
# bbox center loss
cent_loss = nb_anchors_loss_metrics.cent_L(pred,*y,gaf=gaf)
cent_loss

tensor(3.8638, grad_fn=<DivBackward0>)

In [147]:
# positive confidence loss
pconf_loss = nb_anchors_loss_metrics.pConf_L(pred,*y,gaf=gaf)
pconf_loss

tensor(0.9587, grad_fn=<DivBackward0>)

In [148]:
# negative confidence loss
nconf_loss = nb_anchors_loss_metrics.nConf_L(pred,*y,gaf=gaf)
nconf_loss

tensor(0.6954, grad_fn=<DivBackward0>)

In [ ]:
# bbox height and width loss
#网络输出内容里面不再有宽高，所以这个会异常out of range
hw_loss = nb_anchors_loss_metrics.hw_L(pred,*y,gaf=gaf)
hw_loss

In [ ]:
type(pred)
len(pred)
pred[0].shape

In [ ]:
pred[1].shape

In [ ]:
pred[2].shape

In [149]:
# total loss
total_loss = yolo_L_std(pred,*y, clas_weights=None,gaf=gaf)
total_loss

> <ipython-input-110-b3b853fefeaf>(47)yolo_L_std()
-> gt_t = gaf.b2t(gt_bboxs,idx,eps=1)
(Pdb) c
> <ipython-input-110-b3b853fefeaf>(45)yolo_L_std()
-> import pdb; pdb.set_trace()
(Pdb) c
> <ipython-input-110-b3b853fefeaf>(47)yolo_L_std()
-> gt_t = gaf.b2t(gt_bboxs,idx,eps=1)
(Pdb) c
yolo_l tensor(4.4277, grad_fn=<DivBackward0>) tensor(3.8638, grad_fn=<DivBackward0>) tensor(0.9587, grad_fn=<DivBackward0>) tensor(0.6945, grad_fn=<DivBackward0>)


tensor(9.9448, grad_fn=<AddBackward0>)

In [150]:
clas_loss+cent_loss+pconf_loss+nconf_loss

tensor(9.9457, grad_fn=<AddBackward0>)

In [ ]:
#断言失败.因为nconf_loss去掉了邻居anchor，和原始的nb_anchors_loss_metrics.nConf_L有差别
assert (clas_loss+cent_loss+pconf_loss+nconf_loss)==total_loss

### bbox center difference

In [142]:
#nb_anchors_loss_metrics.cent_d(pred,*y,gaf=gaf)
cent_d_std(pred,*y,gaf=gaf)

tensor(0.0136, grad_fn=<DivBackward0>)

# export

In [151]:
!python notebook2script.py --fname 'anchors_loss_metrics_std.ipynb' --outputDir './exp/'

Converted anchors_loss_metrics_std.ipynb to exp/nb_anchors_loss_metrics_std.py
